In [3]:
import pandas as pd
import os
import openai
from datetime import datetime
import time

In [4]:
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.environ["OPENAI_API_KEY"] # Assume OpenAI API key is stored in the `.env` file

In [5]:
data_dir = '../data/'
output_dir = '../output/'

In [6]:
df = pd.read_csv(os.path.join(data_dir, 'ChatGPT_ERvisit_prompt.csv'))

In [9]:
df.head()

,No,カテゴリ,質問項目,プロンプト,救急受診ガイドの判定
0,1,アルゴリズム,呼吸をしていない。息がない。,あなたは救急医です。救急受診が必要かどうかを判断してください。呼吸をしていない。息がない。はい。,赤
1,2,アルゴリズム,脈がない。心臓が止まっている。,あなたは救急医です。救急受診が必要かどうかを判断してください。脈がない。心臓が止まっている。はい。,赤
2,3,アルゴリズム,沈んでいる。,あなたは救急医です。救急受診が必要かどうかを判断してください。沈んでいる。はい。,赤
3,4,アルゴリズム,冷たくなっている。,あなたは救急医です。救急受診が必要かどうかを判断してください。冷たくなっている。はい。,赤
4,5,アルゴリズム,呼びかけても、反応がない。,あなたは救急医です。救急受診が必要かどうかを判断してください。呼びかけても、反応がない。はい。,赤


In [5]:
def obtain_response(question: str):
    start_time = datetime.now()
    for _ in range(3):  # Retry up to 3 times
        try:
            response = openai.ChatCompletion.create(
                model='gpt-3.5-turbo',
                messages=[
                    {
                        'role': 'user',
                        'content': question
                    },
                ],
                logprobs = 0, # Do not store conversation history in OpenAI's system
            )
            time_diff = round((datetime.now() - start_time).total_seconds())
            response_text = response.choices[0]['message']['content'].strip()
            return response_text, time_diff
        except openai.error.OpenAIError as e:
            print(f"OpenAI error: {e}")
            time.sleep(1)  # Wait for 1 second before retrying
        except Exception as e:
            print(f"Other error: {e}")
            time.sleep(1)  # Wait for 1 second before retrying
    else:
        print(f"Failed to get a response for question: {question}")
        return None, None

In [6]:
# Shuffle question order
df = df.sample(frac=1).reset_index(drop=True)

numbers = []
questions = []
responses = []
times = []

for index, row in df.iterrows():
    question = str(row['プロンプト'])
    response, time_diff = obtain_response(question)
    numbers.append(row['No'])
    questions.append(question)
    responses.append(response)
    times.append(time_diff)

OpenAI error: HTTP code 502 from API (<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>cloudflare</center>
</body>
</html>
)


In [9]:
df_answer = pd.DataFrame({
    'No': numbers,
    'プロンプト': questions,
    '回答': responses,
    '回答時間(秒)': times
})

In [8]:
df_answer.to_csv(os.path.join(output_dir, 'test_random50_ChatGPT_answer.csv'), encoding='utf-8-sig', index=False)